Frozen Lake
===
The goal of this computer assignment is to get familiar with OpenAI Gym, implement value iteration and policy iteration.

Problem Description
---
OpenAI Gym is a toolkit for developing and comparing reinforcement learning algorithms. It supports teaching agents everything from walking to playing games like Pong or Pinball. For more information visit https://gym.openai.com.

In this computer assigment, you'll get familiar with Frozen Lake environment and implement value and policy iteration algorithms. Frozen Lake is an environment where the agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile. For more information please visit https://gym.openai.com/envs/FrozenLake8x8-v0/.

Your Job
---
1. Get started with gym by following the steps here https://gym.openai.com/docs/.
2. Read https://gym.openai.com/envs/FrozenLake8x8-v0/ to get familiar with the environment, states, reward function, etc.
3. Implement the $\texttt{value_iteration}$ function below.
4. Implement the $\texttt{policy_iteration}$ function below.
5. Answer the questions (By double click on the cell you can edit the cell and put your answer below each question).

In [9]:
from __future__ import print_function
from __future__ import division

import numpy as np
import gym
from gym import wrappers
import time

def run_episode(env, policy, gamma, render = False):
    """ Evaluates policy by using it to run an episode and finding its
    total reward.
    args:
    env: gym environment.
    policy: the policy to be used.
    gamma: discount factor.
    render: boolean to turn rendering on/off.
    returns:
    total reward: real value of the total reward recieved by agent under policy.
    """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma,  n = 100):
    """ Evaluates a policy by running it n times.
    returns:
    average total reward
    """
    scores = [
            run_episode(env, policy, gamma=gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy


def value_iteration(env, gamma, epsilon=1e-20, max_iterations=100000):
    """
    This function implements value iteration algorithm for the infinite
    horizon discounted MDPs. If the sup norm of v_k - v_{k-1} is less than
    epsilon or number of iterations reaches max_iterations, it should return
    the value function.
    """
    start = time.time()
    v = np.zeros(env.nS)  # initialize value-function
    ########################### Your Code Here ###########################
    # Hint: see implementation of extract_policy
    for i in range(max_iterations):
        prv_v = np.copy(v)
        for s in range(env.nS):
            q_sa = [sum([p*(r + gamma * prv_v[s_]) for p, s_, r, _ in env.P[s][a]]) for a in range(env.nA)] 
            v[s] = max(q_sa)
        if (np.linalg.norm((prv_v - v),np.inf) <= epsilon):
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    
   
    
    
    
    
    
    
    
    ########################### End of your code #########################
    end = time.time()
    print("Value iteration took {0} seconds.".format(end - start))
    return v

if __name__ == '__main__':
    np.random.seed(1111)
    env_name  = 'FrozenLake8x8-v0'
    for gamma in [.9, .95, .99, .9999, 1]:
        print("-"*10, "Gamma={0}".format(gamma) ,"-"*10)
        env = gym.make(env_name)
        optimal_v = value_iteration(env, gamma);
        policy = extract_policy(optimal_v, gamma)
        policy_score = evaluate_policy(env, policy, gamma, n=1000)
        print('Average score = ', policy_score)

---------- Gamma=0.9 ----------
Value-iteration converged at iteration# 315.
Value iteration took 0.641289234161377 seconds.
Average score =  0.00628743678838866
---------- Gamma=0.95 ----------
Value-iteration converged at iteration# 497.
Value iteration took 0.7956991195678711 seconds.
Average score =  0.0479452941660768
---------- Gamma=0.99 ----------
Value-iteration converged at iteration# 1126.
Value iteration took 1.6570303440093994 seconds.
Average score =  0.4065790712237092
---------- Gamma=0.9999 ----------
Value-iteration converged at iteration# 2336.
Value iteration took 3.995661735534668 seconds.
Average score =  0.865706696499763
---------- Gamma=1 ----------
Value-iteration converged at iteration# 2357.
Value iteration took 3.9234561920166016 seconds.
Average score =  0.866


Policy Iteration
---

In [10]:
def compute_policy_v(env, policy, gamma):
    """ Iteratively evaluate the value-function under policy.
    Alternatively, we could formulate a set of linear equations in iterms of v[s] 
    and solve them to find the value function.
    """
    v = np.zeros(env.nS)
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(env.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

def policy_iteration(env, gamma, max_iterations=100000):
    """
    This function implements policy iteration algorithm.
    """
    start = time.time()
    policy = np.random.choice(env.nA, size=(env.nS))  # initialize a random policy
    ########################### Your Code Here ###########################
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    
    
    
    
    
    
    
    
    
    ########################### End of your code #########################
    end = time.time()
    print("Policy iteration took {0} seconds.".format(end - start))
    return policy


if __name__ == '__main__':
    np.random.seed(1111)
    env_name  = 'FrozenLake8x8-v0'
    for gamma in [.9, .95, .99, .9999, 1]:
        print("-"*10, "Gamma={0}".format(gamma) ,"-"*10)
        env = gym.make(env_name)
        optimal_policy = policy_iteration(env, gamma=gamma)
        scores = evaluate_policy(env, optimal_policy, gamma=gamma)
        print('Average scores = ', np.mean(scores))

---------- Gamma=0.9 ----------
Policy-Iteration converged at step 5.
Policy iteration took 0.4083437919616699 seconds.
Average scores =  0.00594476566695997
---------- Gamma=0.95 ----------
Policy-Iteration converged at step 3.
Policy iteration took 0.44911885261535645 seconds.
Average scores =  0.048241249527001685
---------- Gamma=0.99 ----------
Policy-Iteration converged at step 8.
Policy iteration took 2.030360221862793 seconds.
Average scores =  0.37057608411841303
---------- Gamma=0.9999 ----------
Policy-Iteration converged at step 12.
Policy iteration took 7.533506870269775 seconds.
Average scores =  0.861386129235637
---------- Gamma=1 ----------
Policy-Iteration converged at step 6.
Policy iteration took 4.60036563873291 seconds.
Average scores =  0.88


Questions
--- 

#### 1. How many iterations did it take for the value iteration to converge? How about policy iteration?
For gamma=1, value iteration converges in 2357 steps & policy iteration converges in 6 steps. For gamma=0.9, value iteration converges in 315 steps & policy iteration converges in 5 steps. Number of iterations are printed out for every gamma value & iterative method.

#### 2. How much time did it take for the value iteration to converge? How about the policy iteration?
For gamma=1, 4.749seconds for value iteration & 3.478seconds for policy iteration. For different gamma values & algorithm, time required for convergence is printed.

#### 3. Which algorithm is faster? Why?
Policy iteration algorithm is faster because it converges in finite time since it finds optimal policy directly unlike value iteration algorithm which converges asymptotically. Value iteration algorithm tries to find optimal value function from which we need to find the optimal policy.

#### 4. How does the average score change as $\gamma$ gets closer to 1? Why?
For gamma =1, average score of 0.864 for value iteration & 0.88 for policy iteration. The average score changes drastically(increases) when gamma approches 1 beacause the rewards of high step_idx is taken as it is without multiplying by gamma to the power of (num of step_idx). 